# Задача 2
## Сервис поиска студентов на базе технологии XML-RPC 

Используем базу на Postgres 16 созданную в задаче 1

In [1]:
%%capture
!pip install psycopg2 openai ipython-sql python-dotenv

In [1]:
from dotenv import load_dotenv
import os

# Загрузка переменных окружения
load_dotenv()

# Получение значений из .env файла
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

# Формирование строки подключения
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

print("Строка подключения успешно создана.")


Строка подключения успешно создана.


In [3]:
%%capture
import json
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData, Column, Integer, String, ForeignKey, TIMESTAMP, JSON
from sqlalchemy.ext.declarative import declarative_base
from dotenv import load_dotenv
from sqlalchemy.orm import sessionmaker
import os

# Загружаем переменные окружения
load_dotenv()

# Получаем параметры подключения
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

# Создаем подключение к базе данных
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
# Создаем сессию
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

## Подгрузка студентов ( ранее добавленные удалил поскольку понадобились дополнительные поля)

In [4]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from models import Student
import json

# Подключение к базе
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()

# Очистка таблицы (если нужно)
session.execute(text("TRUNCATE TABLE students RESTART IDENTITY CASCADE"))
session.commit()

# Загрузка данных
with open('students_update.json', 'r', encoding='utf-8') as f:
    students_data = json.load(f)

for data in students_data:
    try:
        student = Student(
            full_name=data["full_name"],
            age=data["age"],
            gender=data["gender"],
            attendance=data["attendance"],
            grades=data["grades"],
            info=data.get("info", {})
        )
        session.add(student)
    except KeyError as e:
        print(f"Ошибка в данных: отсутствует поле {e}")

try:
    session.commit()
    print(f"Загружено студентов: {len(students_data)}")
except Exception as e:
    session.rollback()
    print(f"Ошибка: {e}")
finally:
    session.close()

Загружено студентов: 10


In [4]:
# Закрытие текущей сессии
session.close()
# Пересоздание engine и сессии
engine.dispose()  # освобождает пул соединений
Session = sessionmaker(bind=engine)
session = Session()

engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()



In [5]:
%%capture
!pip install --upgrade sqlalchemy #обновление sqlalchemy

### Сервер XML-RPC

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import JSONB 
from models import Student, Base  
import threading
import logging


engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()

logging.basicConfig(level=logging.INFO)
# XML-RPC сервер

server = SimpleXMLRPCServer(("localhost", 8000), allow_none=True)
server_thread = threading.Thread(target=server.serve_forever)
server_thread.daemon = True  # Сервер остановится при завершении основного потока
server_thread.start()

print("Сервер запущен на порту 8000 в фоновом режиме.")

def search_students(filters, sort_field="id", sort_order="asc", page=1, per_page=10):
    session = Session()
    logging.info(f"Запрос: {filters}")
    try:
        query = session.query(Student)
        
        # Фильтрация
        if "full_name" in filters:
            query = query.filter(
                func.to_tsvector('russian', Student.full_name).op("@@")(
                    func.websearch_to_tsquery('russian', filters["full_name"])
                )
            )
        if "age" in filters:
            query = query.filter(Student.age == filters["age"])
        if "gender" in filters:
            query = query.filter(Student.gender == filters["gender"])
        if "lecture_presence" in filters:
            query = query.filter(
                Student.attendance.has_key(filters["lecture_presence"]["lecture"])
            ).filter(
                Student.attendance[filters["lecture_presence"]["lecture"]].astext == str(filters["lecture_presence"]["is_present"])
            )
        
        # Сортировка
        if sort_order == "desc":
            query = query.order_by(getattr(Student, sort_field).desc())
        else:
            query = query.order_by(getattr(Student, sort_field).asc())
            
        # Пагинация
        total = query.count()
        students = query.offset((page-1)*per_page).limit(per_page).all()
        
        # Результат
        return {
            "data": [
                {
                    "id": s.id,
                    "full_name": s.full_name,
                    "age": s.age,
                    "gender": s.gender,
                    "attendance": s.attendance,
                    "grades": s.grades
                } for s in students
            ],
            "meta": {"page": page, "per_page": per_page, "total": total}
        }
    except Exception as e:
        logging.error(f"Ошибка: {e}")
        raise

    finally:
        session.close()

server.register_function(search_students, "search_students")

if __name__ == "__main__":
    server.serve_forever()

    # код перенесен в файл server.py
    # запускать в отдельном терминале 
   # python server.py
   # lsof -i :8000  # проверка занятости порта
   # kill -9 pid 

INFO:root:Запрос: {'full_name': 'Петров'}
127.0.0.1 - - [31/Mar/2025 22:05:43] "POST / HTTP/1.1" 200 -
INFO:root:Запрос: {'full_name': 'Петров'}
127.0.0.1 - - [31/Mar/2025 22:06:27] "POST / HTTP/1.1" 200 -
INFO:root:Запрос: {'full_name': 'Петрова'}
127.0.0.1 - - [31/Mar/2025 22:06:43] "POST / HTTP/1.1" 200 -
INFO:root:Запрос: {'gender': 'male', 'lecture_presence': {'lecture': 'Основы ML', 'is_present': True}}
127.0.0.1 - - [31/Mar/2025 22:15:11] "POST / HTTP/1.1" 200 -
INFO:root:Запрос: {'gender': 'male', 'lecture_presence': {'lecture': '1', 'is_present': True}}
127.0.0.1 - - [31/Mar/2025 22:18:03] "POST / HTTP/1.1" 200 -
INFO:root:Запрос: {'gender': 'male', 'lecture_presence': {'lecture': 'Лекция 1', 'is_present': True}}
127.0.0.1 - - [31/Mar/2025 22:18:32] "POST / HTTP/1.1" 200 -
INFO:root:Запрос: {'gender': 'male', 'lecture_presence': {'lecture': 'Лекция 1', 'is_present': True}}
127.0.0.1 - - [31/Mar/2025 22:20:54] "POST / HTTP/1.1" 200 -
INFO:root:Запрос: {'full_name': 'Петрова'}
1

In [8]:
# Тест подключения к серверу
import xmlrpc.client
proxy = xmlrpc.client.ServerProxy("http://localhost:8000/")
print(proxy.search_students({"full_name": "Петрова"}))

{'data': [{'id': 2, 'full_name': 'Петрова Анна Сергеевна', 'age': 21, 'gender': 'женский', 'attendance': {'Лекция 1': True, 'Семинар 2': True}, 'grades': {'Базы данных': 5, 'Программирование': 5}}], 'meta': {'page': 1, 'per_page': 10, 'total': 1}}


In [12]:
proxy = xmlrpc.client.ServerProxy("http://localhost:8000/")

# Аргументы передаются позиционно: 
# (filters, sort_field, sort_order, page, per_page)
try:
    response = proxy.search_students(
        {
            "gender": "мужской",
            "lecture_presence": {"lecture": "Лекция 1", "is_present": True}
        },
        "age",  # sort_field
        "desc", # sort_order
        1,      # page
        5       # per_page
    )

    print(response)
except xmlrpc.client.Fault as e:
    print(f"Ошибка сервера: {e}")

{'data': [{'id': 7, 'full_name': 'Федоров Денис Игоревич', 'age': 25, 'gender': 'мужской', 'attendance': {'Лекция 1': True, 'Семинар 2': True}, 'grades': {'Физика': 4, 'Информатика': 5}}, {'id': 5, 'full_name': 'Новиков Павел Сергеевич', 'age': 24, 'gender': 'мужской', 'attendance': {'Лекция 1': True, 'Семинар 2': False}, 'grades': {'Философия': 4, 'Экономика': 5}}, {'id': 1, 'full_name': 'Иванов Иван Иванович', 'age': 22, 'gender': 'мужской', 'attendance': {'Лекция 1': True, 'Семинар 2': False}, 'grades': {'Физика': 4, 'Математика': 5}}], 'meta': {'page': 1, 'per_page': 5, 'total': 3}}


In [ ]:
## Добавлены переносы строк и команды для перехода по результатам, код вынесен для запуска из client.py

import xmlrpc.client
import os

# Адрес XML-RPC сервера
proxy = xmlrpc.client.ServerProxy("http://localhost:8000/")

# Настройки запроса
filters = {
   # "gender": "мужской",
   # "lecture_presence": {"lecture": "Лекция 1", "is_present": True}
}
sort_field = "age"
sort_order = "desc"
per_page = 5


def clear_terminal():
    os.system("cls" if os.name == "nt" else "clear")


def print_students(response):
    clear_terminal()
    data = response["data"]
    meta = response["meta"]
    total_pages = ((meta["total"] - 1) // meta["per_page"]) + 1

    print(f"\nСтраница: {meta['page']} из {total_pages} | Всего записей: {meta['total']}\n")

    if not data:
        print("Нет данных на этой странице.")
        return

    start_index = (meta['page'] - 1) * meta['per_page'] + 1
    for i, student in enumerate(data, start=start_index):
        print(f"Студент {i}:")
        print(f"  ID: {student['id']}")
        print(f"  ФИО: {student['full_name']}")
        print(f"  Возраст: {student['age']}")
        print(f"  Пол: {student['gender']}")
        print("  Посещаемость:")
        for lecture, present in student["attendance"].items():
            print(f"    - {lecture}: {'Присутствовал' if present else 'Отсутствовал'}")
        print("  Оценки:")
        for subject, grade in student["grades"].items():
            print(f"    - {subject}: {grade}")
        print("-" * 40)


def main():
    page = 1
    while True:
        try:
            response = proxy.search_students(filters, sort_field, sort_order, page, per_page)
            print_students(response)
        except xmlrpc.client.Fault as e:
            print(f"Ошибка сервера: {e}")
            break
        except Exception as e:
            print(f"Ошибка: {e}")
            break

        cmd = input("\n[n] — следующая | [p] — предыдущая | [q] — выход: ").strip().lower()
        if cmd == "n":
            if page * per_page < response["meta"]["total"]:
                page += 1
            else:
                print("Это последняя страница.")
        elif cmd == "p":
            if page > 1:
                page -= 1
            else:
                print("Это первая страница.")
        elif cmd == "q":
            print("Выход.")
            break
        else:
            print("Неверная команда.")


if __name__ == "__main__":
    main()



Страница: 1 из 2 | Всего записей: 10

👤 Студент 1:
  ID: 7
  ФИО: Федоров Денис Игоревич
  Возраст: 25
  Пол: мужской
  Посещаемость:
    - Лекция 1: Присутствовал
    - Семинар 2: Присутствовал
  Оценки:
    - Физика: 4
    - Информатика: 5
----------------------------------------
👤 Студент 2:
  ID: 5
  ФИО: Новиков Павел Сергеевич
  Возраст: 24
  Пол: мужской
  Посещаемость:
    - Лекция 1: Присутствовал
    - Семинар 2: Отсутствовал
  Оценки:
    - Философия: 4
    - Экономика: 5
----------------------------------------
👤 Студент 3:
  ID: 3
  ФИО: Сидоров Алексей Петрович
  Возраст: 23
  Пол: мужской
  Посещаемость:
    - Лекция 1: Отсутствовал
    - Семинар 2: Присутствовал
  Оценки:
    - Химия: 3
    - История: 4
----------------------------------------
👤 Студент 4:
  ID: 1
  ФИО: Иванов Иван Иванович
  Возраст: 22
  Пол: мужской
  Посещаемость:
    - Лекция 1: Присутствовал
    - Семинар 2: Отсутствовал
  Оценки:
    - Физика: 4
    - Математика: 5
-----------------------------

## Проверка производительности сервиса

In [1]:
%%capture
!pip install locust

### Код для запуска тестирования с помощью сервиса locust вынесен в файл locust_test.py
### Запуск из консоли:
```
locust -f locust_test.py
```

### Запуск под одним пользователем
![locust1.png](./tasks_1-2/images/locust1.png "1")

### Запуск под 10 пользователями
![locust10.png](./tasks_1-2/images/locust10.png "10")

### Документирование сервиса xml-rpc

In [5]:
import xmlrpc.client

# Подключение к серверу
proxy = xmlrpc.client.ServerProxy("http://localhost:8000/")

# Получение списка методов
methods = proxy.system.listMethods()
print("Доступные методы:", methods)

# Документирование каждого метода
for method in methods:
    try:
        help_text = proxy.system.methodHelp(method)
        signature = proxy.system.methodSignature(method)
        print(f"\nМетод: {method}")
        print(f"Сигнатура: {signature}")
        print(f"Описание: {help_text}")
    except Exception as e:
        print(f"Ошибка для метода {method}: {e}")

Доступные методы: ['search_students', 'system.listMethods', 'system.methodHelp', 'system.methodSignature']

Метод: search_students
Сигнатура: signatures not supported
Описание: Поиск студентов по фильтру.
Параметры:
- filter: dict (например, {"gender": "мужской"}),
- sort_field: str (поле сортировки),
- sort_order: str ("asc" или "desc"),
- page: int (номер страницы),
- per_page: int (количество на странице).

Метод: system.listMethods
Сигнатура: signatures not supported
Описание: system.listMethods() => ['add', 'subtract', 'multiple']

Returns a list of the methods supported by the server.

Метод: system.methodHelp
Сигнатура: signatures not supported
Описание: system.methodHelp('add') => "Adds two integers together"

Returns a string containing documentation for the specified method.

Метод: system.methodSignature
Сигнатура: signatures not supported
Описание: system.methodSignature('add') => [double, int, int]

Returns a list describing the signature of the method. In the
above exampl